# Assignment filter


ojnsdvojsdnjdg

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
from pathlib import Path

import sympy as sp
import numpy as np
import xarray as xr
import xarray.ufuncs as xf
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib widget

from cw.filters import IteratedExtendedKalmanFilter
from system_identification.load_assignment_data import load_assignment_data

In [3]:
u, v, w, cau = sp.symbols("u, v, w, C_alpha_u")
alpha_m, beta_m, v_m = sp.symbols("alpha_m, beta_m, V_m")
u_dot, v_dot, w_dot = sp.symbols("udot, vdot, wdot")

In [4]:
iekf = IteratedExtendedKalmanFilter(
    x=[u, v, w, cau],
    z=[alpha_m, beta_m, v_m],
    u=[u_dot, v_dot, w_dot],
    f=sp.Matrix([
        u_dot,
        v_dot,
        w_dot,
        0.
    ]),
    h=sp.Matrix([
        sp.atan(w / u) * (1 + cau),
        sp.atan(v / sp.sqrt(u**2 + w**2)),
        sp.sqrt(u**2 + v**2 + w**2)
    ]),
    g=sp.Matrix(np.diag([1, 1, 1, 0])),
    max_iterations=100,
    eps=1e-10,
)
iekf

$$x = \left( u, \  v, \  w, \  C_{\alpha u}\right)$$  
$$z = \left( \alpha_{m}, \  \beta_{m}, \  V_{m}\right)$$  
$$u = \left( \dot{u}, \  \dot{v}, \  \dot{w}\right)$$  
$$f(\dots) = \left[\begin{matrix}\dot{u}\\\dot{v}\\\dot{w}\\0.0\end{matrix}\right]$$  
$$g(\dots) = \left[\begin{matrix}1 & 0 & 0 & 0\\0 & 1 & 0 & 0\\0 & 0 & 1 & 0\\0 & 0 & 0 & 0\end{matrix}\right]$$  
$$h(\dots) = \left[\begin{matrix}\left(C_{\alpha u} + 1\right) \operatorname{atan}{\left(\frac{w}{u} \right)}\\\operatorname{atan}{\left(\frac{v}{\sqrt{u^{2} + w^{2}}} \right)}\\\sqrt{u^{2} + v^{2} + w^{2}}\end{matrix}\right]$$  
$$F_x(\dots) = \left[\begin{matrix}0 & 0 & 0 & 0\\0 & 0 & 0 & 0\\0 & 0 & 0 & 0\\0 & 0 & 0 & 0\end{matrix}\right]$$  
$$H_x(\dots) = \left[\begin{matrix}- \frac{w \left(C_{\alpha u} + 1\right)}{u^{2} \left(1 + \frac{w^{2}}{u^{2}}\right)} & 0 & \frac{C_{\alpha u} + 1}{u \left(1 + \frac{w^{2}}{u^{2}}\right)} & \operatorname{atan}{\left(\frac{w}{u} \right)}\\- \frac{u v}{\left(u^{2} + w^{2}\right)^{\frac{3}{2}} \left(\frac{v^{2}}{u^{2} + w^{2}} + 1\right)} & \frac{1}{\sqrt{u^{2} + w^{2}} \left(\frac{v^{2}}{u^{2} + w^{2}} + 1\right)} & - \frac{v w}{\left(u^{2} + w^{2}\right)^{\frac{3}{2}} \left(\frac{v^{2}}{u^{2} + w^{2}} + 1\right)} & 0\\\frac{u}{\sqrt{u^{2} + v^{2} + w^{2}}} & \frac{v}{\sqrt{u^{2} + v^{2} + w^{2}}} & \frac{w}{\sqrt{u^{2} + v^{2} + w^{2}}} & 0\end{matrix}\right]$$

In [5]:
data_dir_path = Path().cwd().parent.parent / "assignment"
data = load_assignment_data(data_dir_path)
data

<xarray.Dataset>
Dimensions:  (t: 10001)
Coordinates:
  * t        (t) float64 0.0 0.01 0.02 0.03 0.04 ... 99.97 99.98 99.99 100.0
Data variables:
    alpha_m  (t) float64 -0.004572 -0.003229 -0.004428 ... -0.2303 -0.2288
    beta_m   (t) float64 0.002069 0.002634 0.0005577 ... 0.2579 0.2543 0.2574
    V_m      (t) float64 144.9 147.6 153.4 148.4 ... 152.3 152.1 152.1 153.9
    udot     (t) float64 1.043e-05 6.843e-06 4.02e-06 ... -2.244 -2.153 -2.05
    vdot     (t) float64 0.45 0.4499 0.4497 0.4494 ... 24.86 24.46 24.05 23.63
    wdot     (t) float64 2.855e-07 0.0225 0.04499 ... -49.15 -46.12 -43.07
    c_m      (t) float64 -0.05797 -0.05873 -0.05849 ... -0.03054 -0.02537

In [6]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data.alpha_m, data.beta_m, data.c_m)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
f_data = iekf.filter(
    data=data,
    x_0=[150, 0, 0, 0.6021],
#     p_0=np.diag([1.e-3, 1., 1., 1.]),
    p_0=np.array([[ 1.86026173e-06,  8.12747809e-08, -1.76199928e-06, 3.52600421e-08],
                  [ 8.12747809e-08,  4.28292040e-05,  4.73120858e-08, 1.29804829e-10],
                  [-1.76199928e-06,  4.73120858e-08,  1.88365617e-05, -2.30419525e-07],
                  [ 3.52600421e-08,  1.29804829e-10, -2.30419525e-07, 1.45268357e-07]]),
    q=np.diag([1e-3**2, 1e-3**2, 1e-3**2, 0.**2]),
    r=np.diag([0.01**2, 0.0058**2, 0.112**2]),
    verbose=True
)
f_data["vm_est"] = xf.sqrt(xf.square(f_data.u_est) + xf.square(f_data.v_est) + xf.square(f_data.w_est))

  0%|          | 0/10000 [00:00<?, ?it/s]

In [15]:
f_data

<xarray.Dataset>
Dimensions:        (dim_0: 4, dim_1: 4, t: 10001)
Coordinates:
  * t              (t) float64 0.0 0.01 0.02 0.03 ... 99.97 99.98 99.99 100.0
Dimensions without coordinates: dim_0, dim_1
Data variables:
    alpha_m        (t) float64 -0.004572 -0.003229 -0.004428 ... -0.2303 -0.2288
    beta_m         (t) float64 0.002069 0.002634 0.0005577 ... 0.2543 0.2574
    V_m            (t) float64 144.9 147.6 153.4 148.4 ... 152.1 152.1 153.9
    udot           (t) float64 1.043e-05 6.843e-06 4.02e-06 ... -2.153 -2.05
    vdot           (t) float64 0.45 0.4499 0.4497 0.4494 ... 24.46 24.05 23.63
    wdot           (t) float64 2.855e-07 0.0225 0.04499 ... -49.15 -46.12 -43.07
    c_m            (t) float64 -0.05797 -0.05873 -0.05849 ... -0.03054 -0.02537
    u_est          (t) float64 150.0 150.0 150.0 150.0 ... 149.6 149.5 149.5
    v_est          (t) float64 0.0 0.004484 0.008989 ... 38.0 38.24 38.48
    w_est          (t) float64 0.0 0.000703 0.001262 ... -20.59 -21.08 -21.54
    C_alpha_u_est  (t) float64 0.6021 0.6021 0.6021 ... 0.6044 0.6044 0.6043
    p_k1k1         (t, dim_0, dim_1) float64 1.86e-06 8.127e-08 ... 7.264e-08
    iekf_i_count   (t) int64 0 3 2 3 2 2 3 3 2 2 3 3 ... 2 2 3 2 3 2 3 3 3 3 3 3
    vm_est         (t) float64 150.0 150.0 150.0 150.0 ... 155.7 155.8 155.9

In [12]:
f_data.p_k1k1.isel(t=-1).values

array([[ 1.86026173e-06,  8.12747809e-08, -1.76199928e-06,
         3.52600421e-08],
       [ 8.12747809e-08,  4.28292040e-05,  4.73120858e-08,
         1.29804829e-10],
       [-1.76199928e-06,  4.73120858e-08,  1.88365617e-05,
        -2.30419525e-07],
       [ 3.52600421e-08,  1.29804829e-10, -2.30419525e-07,
         1.45268357e-07]])

In [16]:
plt.figure()
# f_data.u.plot()
f_data.C_alpha_u_est.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<Figure size 640x480 with 0 Axes>

In [17]:
plt.figure()
f_data.u_est.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<Figure size 640x480 with 0 Axes>

In [18]:
plt.figure()
f_data.V_m.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<Figure size 640x480 with 0 Axes>

In [19]:
plt.figure()
f_data.V_m.plot()
f_data.vm_est.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<Figure size 640x480 with 0 Axes>

In [20]:
plt.figure()
f_data.u_est.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<Figure size 640x480 with 0 Axes>

In [19]:
# !jupyter nbconvert --to script assignment_cw_iekf.ipynb

[NbConvertApp] Converting notebook assignment_cw_iekf.ipynb to script
[NbConvertApp] Writing 2333 bytes to assignment_cw_iekf.py
